# Correlation Power Analysis
Submitters:
1. Chen Frydman 208009845
2. Hadi Shaheen 315490193

```bash
docker pull annakul1/attacks_on_implementations:Assignment2
docker run -p 80:8080 annakul1/attacks_on_implementations:Assignment2
```

# Imports

In [1]:
import requests
import numpy as np
import time
import os
import concurrent.futures

# Parameters

In [2]:
USER = 208009845
DIFFICULTY = 1
BASE_URL = "aoi-assignment2.oy.ne.ro"
MAX_PLAINTEXT_LENGTH = 32
MAX_KEY_LENGTHH = 32
AMOUNT_OF_TRACES = 10000
TIME_LIMIT = 10 * 60 # 10 minutes in seconds
DIFFICULTY_TO_DOWNLOAD = 20
KEY_LENGTH = 16
PREFIX_TRACES_FILE_NAME = f"traces_{USER}"
PREFIX_PLAINTEXT_FILE_NAME = f"plaintext_{USER}"
PREFIX_ENCRYPT_URL = rf"http://{BASE_URL}:8080/encrypt?user={USER}"
PREFIX_VERIFY_URL = rf"http://{BASE_URL}:8080/verify?user={USER}"

AesSbox = [
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76,
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0,
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15,
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75,
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84,
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf,
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8,
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2,
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73,
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb,
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79,
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08,
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a,
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e,
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf,
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16
]

print(f"{PREFIX_ENCRYPT_URL}&difficulty={DIFFICULTY}")

http://aoi-assignment2.oy.ne.ro:8080/encrypt?user=208009845&difficulty=1


# Crack the key

In [3]:
def download_power_traces(
    difficulty: int = DIFFICULTY,
    prefix_traces_filename: str = PREFIX_TRACES_FILE_NAME,
    prefix_plaintext_filename: str = PREFIX_PLAINTEXT_FILE_NAME,
    serverURL: str = PREFIX_ENCRYPT_URL,
    number_of_power_traces: int = AMOUNT_OF_TRACES,
):
    """
    Downloads power traces and corresponding plaintexts from the remote server and saves them to files.

    Args:
        prefix_traces_filename (str): Prefix for the power traces file name.
        prefix_plaintext_filename (str): Prefix for the plaintexts file name.
        serverURL (str): The base URL of the encryption server.
        number_of_power_traces (int): Number of power traces to download.
        difficulty (int): The difficulty level for the challenge.

    Saves:
        - Power traces to 'files/{prefix_traces_filename}_{difficulty}_{number_of_power_traces}.txt'
        - Plaintexts to 'files/{prefix_plaintext_filename}_{difficulty}_{number_of_power_traces}.txt'
    """
    os.makedirs("files", exist_ok=True)
    traces_filename = f"{prefix_traces_filename}_{difficulty}_{number_of_power_traces}.txt"
    traces_filepath = os.path.join("files", traces_filename)
    if os.path.exists(traces_filepath):
        print(f"{traces_filepath} already exists. Skipping download.")
        return

    plaintext_filename = f"{prefix_plaintext_filename}_{difficulty}_{number_of_power_traces}.txt"
    plaintext_filepath = os.path.join("files", plaintext_filename)
    if os.path.exists(plaintext_filepath):
        print(f"{plaintext_filepath} already exists. Skipping download.")
        return

    with open(plaintext_filepath, "w") as plaintext_file, open(traces_filepath, "w") as traces_file:
        url = f"{serverURL}&difficulty={difficulty}"
        print(url)
        for _ in range(number_of_power_traces):
            resp = requests.get(url)
            data = resp.json()
            plaintext = data["plaintext"]
            plaintext_file.write("".join(map(str, plaintext)) + "\n")
            leaks = data["leaks"]
            traces_file.write(" ".join(map(str, leaks)) + "\n")



def getMeansVariances(filename: str):
    """
    Reads the file containing saved power traces and calculates the mean and variance
    of each position in the trace.

    Args:
        filename (str): The name of the file containing the saved power traces.

    Returns:
        tuple: A tuple containing two lists:
            - means (list): A list of means for each position in the trace.
            - variances (list): A list of variances for each position in the trace.
    """
    if not os.path.exists(filename):
        raise FileNotFoundError(f"{filename} does not exist. Please download the power traces first.")

    data = np.loadtxt(filename)
    means = np.mean(data, axis=0)
    variances = np.var(data, axis=0)
    print("Mean\tVariance")
    for m, v in zip(means, variances):
        print(f"{m}\t{v}")
    return means.tolist(), variances.tolist()

def hamming_weight(n):
    """Calculates the Hamming weight of an integer."""
    return bin(n).count('1')

In [15]:

def find_key(trashhold, plaintext_filename, traces_filename, amount_of_traces: int = AMOUNT_OF_TRACES, key_length: int = KEY_LENGTH):
    # Load plaintexts from plaintext_filename
    plaintexts = []
    with open(plaintext_filename, 'r') as pf:
        for index, line in enumerate(pf):
            if index == amount_of_traces:
                break
            cleaned_line = ''.join(line.strip().split()) 
            hex_bytes = [cleaned_line[i:i+2] for i in range(0, len(cleaned_line), 2)]
            plaintexts.append([int(byte, 16) for byte in hex_bytes])
    
    # Load power traces from traces_filename
    power_traces = []
    with open(traces_filename, 'r') as tf:
        for index, line in enumerate(tf):
            if index == amount_of_traces:
                break
            power_traces.append([float(val) for val in line.strip().split()])
    
    if not power_traces or not plaintexts:
        raise ValueError("Power traces or plaintexts are empty. Please check the input files.")
    
    if not power_traces: 
        raise ValueError("No power traces loaded to determine trace length.")
    trace_length = len(power_traces[0]) 

    # Initialize results containers
    found_key_bytes = [0] * KEY_LENGTH
    best_correlations = [0.0] * KEY_LENGTH

    for byte_index in range(key_length):
        max_correlation = -1.0
        best_key_guess = 0
        for guess_k in range(256):
            predicted_ham_weights = []
            # Ensure we only iterate up to amount_of_traces if plaintexts has enough entries
            for i in range(min(amount_of_traces, len(plaintexts))):
                if byte_index >= len(plaintexts[i]):
                    continue
                intermediate_val = plaintexts[i][byte_index] ^ guess_k
                sbox_output = AesSbox[intermediate_val]
                predicted_ham_weights.append(hamming_weight(sbox_output))
            
            if not predicted_ham_weights:
                continue

            current_trace_point_correlations = []
            
            for trace_point_idx in range(trace_length): 
                # Ensure we only access traces that exist and have the given trace_point_idx
                actual_power_values_at_point = [
                    trace[trace_point_idx] for trace in power_traces if trace_point_idx < len(trace)
                ]

                # Only proceed if there are enough actual power values for correlation
                if len(actual_power_values_at_point) != len(predicted_ham_weights):
                    # This could happen if some traces are shorter than others
                    continue 

                if np.std(actual_power_values_at_point) == 0 or np.std(predicted_ham_weights) == 0:
                    correlation = 0 
                else:
                    correlation = np.corrcoef(predicted_ham_weights, actual_power_values_at_point)[0, 1]
                
                current_trace_point_correlations.append(abs(correlation))

            if not current_trace_point_correlations:
                continue

            max_corr_for_guess = np.max(current_trace_point_correlations)

            if max_corr_for_guess > max_correlation:
                max_correlation = max_corr_for_guess
                best_key_guess = guess_k
                # Early stop condition - applied to single byte analysis
                if max_correlation > trashhold: # The threshold you defined
                    # print(f"Early stop for byte {byte_index} with correlation {max_correlation:.4f}") # Optional: for debugging
                    print("Early stop,", end=' ')
                    break
        print(f"Byte {byte_index}: Best guess {best_key_guess:02x} with correlation {max_correlation:.4f}")
        found_key_bytes[byte_index] = best_key_guess
        best_correlations[byte_index] = max_correlation


    # Convert the list of byte integers to a hex string
    final_key_hex = ''.join([format(byte, '02x') for byte in found_key_bytes])
    return final_key_hex


# Pre Download the files

In [24]:
for difficulty in range(5, DIFFICULTY_TO_DOWNLOAD + 1):
    download_power_traces(difficulty)
    print(f"Downloaded {AMOUNT_OF_TRACES} traces for difficulty level {difficulty}.")
print(f"Downloaded {AMOUNT_OF_TRACES} traces for each difficulty level up to {DIFFICULTY_TO_DOWNLOAD}.")

http://aoi-assignment2.oy.ne.ro:8080/encrypt?user=208009845&difficulty=5
Downloaded 10000 traces for difficulty level 5.
http://aoi-assignment2.oy.ne.ro:8080/encrypt?user=208009845&difficulty=6
Downloaded 10000 traces for difficulty level 6.
http://aoi-assignment2.oy.ne.ro:8080/encrypt?user=208009845&difficulty=7
Downloaded 10000 traces for difficulty level 7.
http://aoi-assignment2.oy.ne.ro:8080/encrypt?user=208009845&difficulty=8
Downloaded 10000 traces for difficulty level 8.
http://aoi-assignment2.oy.ne.ro:8080/encrypt?user=208009845&difficulty=9
Downloaded 10000 traces for difficulty level 9.
http://aoi-assignment2.oy.ne.ro:8080/encrypt?user=208009845&difficulty=10
Downloaded 10000 traces for difficulty level 10.
http://aoi-assignment2.oy.ne.ro:8080/encrypt?user=208009845&difficulty=11


KeyboardInterrupt: 

# Finding the Keys

In [12]:
def check_if_correct_key(key: str, difficulty: int = DIFFICULTY, url: str = PREFIX_VERIFY_URL):
    url = f"{url}&difficulty={difficulty}&key={key}"
    response = requests.get(url)
    return response.text.strip() == "1"

In [23]:
import math

difficulty = DIFFICULTY
while True:
    plaintext_filename = os.path.join("files", f"{PREFIX_PLAINTEXT_FILE_NAME}_{difficulty}_{AMOUNT_OF_TRACES}.txt")
    traces_filename = os.path.join("files", f"{PREFIX_TRACES_FILE_NAME}_{difficulty}_{AMOUNT_OF_TRACES}.txt")


    #means, variances = getMeansVariances()
    early_stop = 0.9743 * math.exp(-0.286 * difficulty)
    traces = 130 * difficulty - 50
    print(plaintext_filename, traces_filename, traces, early_stop)
    start_time = time.time()
    key = find_key(early_stop , plaintext_filename, traces_filename, traces)
    elapsed_time = time.time() - start_time
    if elapsed_time > TIME_LIMIT:
        print(f"Total runtime: {last_elapsed_time} seconds")
    if not check_if_correct_key(key, difficulty):
        print(f"Key {key} is incorrect for difficulty {difficulty}.")
        break
    print(f"user: {USER},key: {key},difficulty: {difficulty},traces: {traces}, early stopping: {early_stop}, time: {elapsed_time:.2f}")
    difficulty += 1
    last_elapsed_time = elapsed_time


files\plaintext_208009845_1_10000.txt files\traces_208009845_1_10000.txt 80 0.7319551667170511
Early stop, Byte 0: Best guess 77 with correlation 0.8261
Early stop, Byte 1: Best guess 69 with correlation 0.8067
Early stop, Byte 2: Best guess 0e with correlation 0.7836
Early stop, Byte 3: Best guess 57 with correlation 0.7869
Early stop, Byte 4: Best guess 39 with correlation 0.7800
Early stop, Byte 5: Best guess 2d with correlation 0.8467
Early stop, Byte 6: Best guess 53 with correlation 0.8307
Early stop, Byte 7: Best guess ac with correlation 0.8201
Early stop, Byte 8: Best guess 2a with correlation 0.8128
Early stop, Byte 9: Best guess 53 with correlation 0.7890
Early stop, Byte 10: Best guess 12 with correlation 0.7819
Early stop, Byte 11: Best guess cb with correlation 0.7822
Early stop, Byte 12: Best guess ea with correlation 0.7992
Early stop, Byte 13: Best guess 08 with correlation 0.8195
Early stop, Byte 14: Best guess 62 with correlation 0.8237
Byte 15: Best guess 52 with co

FileNotFoundError: [Errno 2] No such file or directory: 'files\\plaintext_208009845_5_10000.txt'